In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
service_fault_codes = pd.read_excel('Service Fault Codes_1_0_0_167.xlsx')
faults = pd.read_csv('J1939Faults.csv', low_memory=False)
diagnostics = pd.read_csv('VehicleDiagnosticOnboardData.csv', low_memory=False)

In [ ]:
# Define the service locations
service_locations = [(36.0666667, -86.4347222), (35.5883333, -86.4438888), (36.1950, -83.174722)]

# Filter out faults occurring within 2 miles of service locations
for loc in service_locations:
    lat_mile = 2/69 # Convert 2 miles to degrees of latitude
    lon_mile = 1/(69*np.cos(np.radians(loc[0]))) * 2 # Convert 2 miles to degrees of longitude at the given latitude
    faults = faults[(np.abs(faults['Latitude'] - loc[0]) > lat_mile) | (np.abs(faults['Longitude'] - loc[1]) > lon_mile)]
faults = faults[faults['EquipmentID'].str.len() <= 5]
faults

In [ ]:
wide_diag = diagnostics.pivot(index='FaultId', columns='Name', values='Value')

In [ ]:
fault_diag = pd.merge(left = faults,
         right = wide_diag,
         left_on = 'RecordID',
         right_on = 'FaultId'
        )
fault_diag

In [ ]:
derates=fault_diag.loc[fault_diag['spn'].isin([5246, 1569])]
full_derate = derates.loc[derates['spn'].isin([5246])]
full_derate

In [ ]:
# Extract unique EquipmentID numbers from full_derate
unique_equip_ids = full_derate['EquipmentID'].unique()

# Filter rows in fault_diag based on unique_equip_ids
fault_diag_filtered = fault_diag[fault_diag['EquipmentID'].isin(unique_equip_ids)]
fault_diag_filtered

In [ ]:
fault_diag_filtered = fault_diag_filtered[fault_diag_filtered['active'] == True]
fault_diag_filtered

In [ ]:
fault_diag_filtered = fault_diag_filtered.sort_values(by=['EquipmentID','EventTimeStamp'], ascending=[True, True])

In [ ]:
fault_diag_filtered = fault_diag_filtered.drop(columns=['ParkingBrake','LocationTimeStamp','active','CruiseControlActive', 'CruiseControlSetSpeed', 'MCTNumber', 'ecuSoftwareVersion', 'ecuSerialNumber','actionDescription', 'faultValue', 'ServiceDistance', 'SwitchedBatteryVoltage'])
fault_diag_filtered

In [ ]:
fault_diag_filtered['EventTimeStamp'] = pd.to_datetime(fault_diag_filtered['EventTimeStamp'])
fault_diag_filtered = fault_diag_filtered.replace(',', '.', regex=True)

In [ ]:
# Create a groupby object using the 'EquipmentID' column
groups = fault_diag_filtered.groupby('EquipmentID')

# Define a function to apply to each group to interpolate missing values using ffill
def interpolate_group(group):
    return group.ffill()

# Apply the function to each group and concatenate the results back into a single DataFrame
interpolated = pd.concat([interpolate_group(group) for _, group in groups])
interpolated

In [ ]:
# Create a groupby object using the 'EquipmentID' column
groups = interpolated.groupby('EquipmentID')

# Define a function to apply to each group to interpolate missing values using ffill
def interpolate_group(group):
    return group.bfill()

# Apply the function to each group and concatenate the results back into a single DataFrame
interpolated_bfill = pd.concat([interpolate_group(group) for _, group in groups])
interpolated_bfill

In [ ]:
interpolated_bfill = interpolated_bfill.bfill()

# Define columns to impute with mode
cols_to_impute = ['ecuModel', 'ecuMake', 'AcceleratorPedal', 'EngineTimeLtd', 'FuelLevel', 'FuelTemperature', 'Throttle']

# Group by EquipmentID and impute missing values with mode
grouped = interpolated_bfill.groupby('EquipmentID')[cols_to_impute].apply(lambda x: x.fillna(x.mode().iloc[0]))

# Update the original DataFrame with imputed values
interpolated_bfill[cols_to_impute] = grouped[cols_to_impute]
interpolated_bfill

In [ ]:
int_cols = ["AcceleratorPedal", "BarometricPressure", "DistanceLtd", "EngineCoolantTemperature", "EngineLoad",
            "EngineOilPressure", "EngineOilTemperature", "EngineRpm", "EngineTimeLtd", "FuelLevel", "FuelLtd",
            "FuelRate", "FuelTemperature", "IntakeManifoldTemperature", "LampStatus", "Speed", "Throttle",
            "TurboBoostPressure"]

# convert columns to int64
interpolated_bfill[int_cols] = interpolated_bfill[int_cols].astype("float")

In [ ]:
equipment_ids_to_drop = ['302', '1878', '1585', '1961', '2007']
interpolated_drop = interpolated_bfill[~interpolated_bfill['EquipmentID'].isin(equipment_ids_to_drop)]
interpolated_drop

In [ ]:
def mileage_to_spn(interpolated_drop):
    result = []
    for eq_id, eq_df in interpolated_drop.groupby('EquipmentID'):
        earliest_mileage = eq_df['DistanceLtd'].iloc[0]
        spn_mileage = eq_df.loc[eq_df['spn'] == 5246, 'DistanceLtd'].iloc[0]
        result.append((eq_id, spn_mileage - earliest_mileage))
    return result

mileage_to_spn_df = pd.DataFrame(mileage_to_spn(interpolated_drop), columns=['EquipmentID', 'mileage_to_spn'])
# calculate the mean excluding rows where value is zero
overall_avg_mileage = mileage_to_spn_df.loc[mileage_to_spn_df['mileage_to_spn'] != 0, 'mileage_to_spn'].mean()
mileage_to_spn_df['overall_avg_mileage'] = overall_avg_mileage

def time_to_spn(interpolated_drop):
    result = []
    for eq_id, eq_df in interpolated_drop.groupby('EquipmentID'):
        earliest_timestamp = eq_df['EventTimeStamp'].iloc[0]
        spn_timestamp = eq_df.loc[eq_df['spn'] == 5246, 'EventTimeStamp'].iloc[0]
        result.append((eq_id, spn_timestamp - earliest_timestamp))
    return result

time_to_spn_df = pd.DataFrame(time_to_spn(interpolated_drop), columns=['EquipmentID', 'time_to_spn'])
overall_avg_time = time_to_spn_df.loc[time_to_spn_df['time_to_spn'] != pd.Timedelta(0), 'time_to_spn'].mean()
time_to_spn_df['overall_avg_time'] = overall_avg_time

def eng_time_to_spn(interpolated_drop):
    result = []
    for eq_id, eq_df in interpolated_drop.groupby('EquipmentID'):
        earliest_eng_time = eq_df['EngineTimeLtd'].iloc[0]
        spn_eng_time = eq_df.loc[eq_df['spn'] == 5246, 'EngineTimeLtd'].iloc[0]
        result.append((eq_id, spn_eng_time - earliest_eng_time))
    return result

eng_time_to_spn_df = pd.DataFrame(eng_time_to_spn(interpolated_drop), columns=['EquipmentID', 'eng_time_to_spn'])
overall_avg_eng_time = eng_time_to_spn_df.loc[eng_time_to_spn_df['eng_time_to_spn'] != 0, 'eng_time_to_spn'].mean()
eng_time_to_spn_df['overall_avg_eng_time'] = overall_avg_eng_time

def fuel_time_to_spn(interpolated_drop):
    result = []
    for eq_id, eq_df in interpolated_drop.groupby('EquipmentID'):
        earliest_fuel_time = eq_df['FuelLtd'].iloc[0]
        spn_fuel_time = eq_df.loc[eq_df['spn'] == 5246, 'FuelLtd'].iloc[0]
        result.append((eq_id, spn_fuel_time - earliest_fuel_time))
    return result

fuel_time_to_spn_df = pd.DataFrame(fuel_time_to_spn(interpolated_drop), columns=['EquipmentID', 'fuel_time_to_spn'])
overall_avg_fuel_time = fuel_time_to_spn_df.loc[fuel_time_to_spn_df['fuel_time_to_spn'] != 0, 'fuel_time_to_spn'].mean()
fuel_time_to_spn_df['overall_avg_fuel_time'] = overall_avg_fuel_time

merged_df = pd.merge(interpolated_drop, mileage_to_spn_df, on='EquipmentID')
merged_df = pd.merge(merged_df, time_to_spn_df, on='EquipmentID')
merged_df = pd.merge(merged_df, eng_time_to_spn_df, on='EquipmentID')
merged_df = pd.merge(merged_df, fuel_time_to_spn_df, on='EquipmentID')
merged_df

In [ ]:
# find the first occurrence of spn = 5246 for each EquipmentID
earliest_timestamps = interpolated_drop.loc[interpolated_drop['spn'] == 5246] \
    .groupby('EquipmentID')['EventTimeStamp'].first()

# create a new column with the time difference in hours between each row and the first occurrence of spn = 5246 for each EquipmentID
interpolated_drop['time_diff'] = (interpolated_drop['EventTimeStamp'] - interpolated_drop['EquipmentID'].map(earliest_timestamps)) \
    .dt.total_seconds() / 3600

# filter the dataframe to keep only the rows where the time difference is less than or equal to 8 hours
filtered_df = interpolated_drop.loc[(interpolated_drop['EquipmentID'].isin(earliest_timestamps.index)) 
    & (interpolated_drop['time_diff'] >= -24)
    & (interpolated_drop['time_diff'] <= 0)]
filtered_df

In [ ]:
filtered_df.groupby(['spn', 'fmi', 'eventDescription']).size().reset_index(name='count').sort_values('count', ascending=False).head(40)

In [ ]:
# Create target column with spn, fmi combos
def check_spn_fmi(row):
    spn_fmi_pairs = [
        (1569, 31), (4094, 18), (3362, 31), (5394, 17),
        (1761, 19), (1761, 9), (5394, 5), (3364, 9),
        (3216, 9), (6802, 31), (3031, 9), (3226, 9),
        (929, 9), (96, 3), (74, 14), (1068, 2),
        (5742, 9), (1787, 11), (5743, 9), (829, 3),
        (3216, 4), (4094, 31)
    ]
    spn = row['spn']
    fmi = row['fmi']
    if (spn, fmi) in spn_fmi_pairs:
        return 1
    else:
        return 0

filtered_df['target'] = filtered_df.apply(check_spn_fmi, axis=1)
filtered_df

In [ ]:
ml_features = filtered_df.drop(columns=['LampStatus','eventDescription','EventTimeStamp','RecordID', 'ESS_Id','EquipmentID','spn','fmi'], axis=1)

In [ ]:
from pycaret.classification import *

# create time series split
split = int(len(ml_features)*0.8)
train_data = ml_features[:split]
test_data = ml_features[split:]

# setup pycaret
clf = setup(data=train_data, target='target', session_id=123, fold_strategy='timeseries')

# train model
#model = create_model('gbc')
#best_model = compare_models()
models_to_exclude = ['lightgbm']

model=compare_models(exclude=models_to_exclude)

# plot feature importance
plot_model(model, plot='feature')